# Computing Pi with Colliding Blocks

This notebook attempts to use colliding blocks to calculate the digits of pi, based off a really cool video by 3Blue1Brown

In [4]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/HEfHFsfGXjs" \
frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" \
allowfullscreen></iframe>

and the paper [Playing Pool with $\pi$](https://www.maths.tcd.ie/~lebed/Galperin.%20Playing%20pool%20with%20pi.pdf) by G. Galperin,

in which a simple problem is described: a block of mass $M$ moves at a constant speed $V$ on a frictionless surface towards a vertical wall. Closer to the wall there is another block of mass $m$ at rest. After the collision, the small block will hit the wall and be reflected back so that the blocks collide again.

If the ratio between the block masses is a power of 100, and the collisions are perfectly elastic, then the total number of collisions $N_\text{collisions}$ (i.e. collisions + reflections) is equal to certain number of digits of $\pi$. More precisely, if the masses satisfy the relation $\frac{M}{m} = 100^{d-1}$, then the number of clacks is equal to the first $d$ digits of $\pi$.

## 1. Below is the javascript i attempted to use to get a nice display for the computation simulation

Only problem is that I wanted to try and use a physics engine for the collision calculations. The first one I used is matter.js, but I learned that they don't have continuous collision detection, so when setting the mass of the larger block to 1000kg, the inner mass goes so fast that the engine no longer detects its collisions, and the block just dissapears... I tried again with planck.js, which is a port of box2d and features ccd, however I get a similar problem, where the inner block goes so fast and begins to clip into the larger block at around 1000kg, throwing off the computation.

In [13]:
%%html
<iframe src="https://sirmammingtonham.github.io/efficientpicomputation/" width="1000" height="1100"></iframe>

## 2. Equations and definitions, according to the paper

### Definitions/Conditions
- when the two blocks hit each other we can count it as a ***collision***
- when the inner block hits the wall we can count it as a ***reflection*** 
- both collisions and reflections are elastic, so energy and momentum are conserved in these processes

We are interested in the total number of "clacks" given by 

\begin{equation} 
N_\text{clacks} = N_\text{collisions} + N_\text{reflections} \tag*{(1)}
\end{equation}

**Pre-condition:** 
- We initialize the simulation with the large block (of mass $M$) moving at some constant speed $V_0$ before colliding with the small block (of mass $m$) initially at rest. 

**Post-condition:** 
- The final state of the simulation will have the two blocks moving independently away from the wall, with the large block moving faster than the small block, guaranteeing no further collisions.

### Equations
**Reflections:** before hitting the wall, the small block moves with positive velocity, which is fully reversed after the collision. This can be proven in general by writting the conservation of total energy and momentum:

\begin{eqnarray}
\frac12 mv^2_\text{before} &=& \frac12 mv^2_\text{after}, \tag*{(2)}\\
mv_\text{before} &=& mv_\text{after}, \tag*{(3)}\\
\end{eqnarray}

and velocity is reversed, so 
\begin{equation} 
v_\text{before} = -v_\text{after} \tag*{(4)}
\end{equation}


**Collisions:** when the blocks collide with one another, they transfer momentum to the other with each collision. Conservation of energy and momentum lead to:

\begin{align*}
\text{conservation of energy  :} \qquad & \frac12 MV^2_k + \frac12 mv^2_k  = \frac12 MV^2_{k+1} + \frac12 mv^2_{k+1}, \tag*{(5)}\\
\text{conservation of momentum  :} \qquad & MV_k - mv_k = MV_{k+1} + mv_{k+1}. \tag*{(6)}\\
\end{align*}

If we define the ratio between the block masses as $\lambda$

\begin{align*}
\lambda = \frac{M}{m} \tag*{(7)}
\end{align*}

and simplify redundant terms, the system of equations can be written as

\begin{align*}
\text{conservation of energy  :} \qquad & \lambda V^2_k + v^2_k = \lambda V^2_{k+1} + v^2_{k+1},
\tag*{(8)} \\
\text{conservation of momentum:} \qquad & \, \lambda V_k - v_k = \lambda V_{k+1} + v_{k+1}.
\tag*{(9)}
\end{align*}

Doing some wolfram alpha magic to solve for $V_{k+1}$ and $v_{k+1}$, we get 
\begin{align}
V_{k+1} &= \left(\frac{\lambda-1}{\lambda+1}\right) V_k - \left(\frac{2}{\lambda+1}\right) v_k \tag*{(10)}, \\
v_{k+1} &= \left(\frac{2\lambda}{\lambda+1}\right) V_k + \left(\frac{\lambda-1}{\lambda+1}\right) v_k. \tag*{(11)}
\end{align}

which leads to a pretty nice computation for our simulation

In [ ]:
import numpy as np

In [28]:
def run_simulation(lambda_=1):
    # lambda_: M/m mass ratio
    k_max = int(10*np.sqrt(lambda_))  # max value for loop
    # create arrays to plot velocity later
    V, v = np.zeros(k_max+1), np.zeros(k_max+1)
    # initial conditions
    V[0], v[0] = 1, 0
    # collide, blocks, collide!
    collisions=0
    for k in range(0, k_max):
        V[k+1] = (lambda_-1)/(lambda_+1) * V[k] - (2)/(lambda_+1) * v[k]         # eq. (12)
        v[k+1] = (2*lambda_)/(lambda_+1) * V[k] + (lambda_-1)/(lambda_+1) * v[k] # eq. (13)
        # declare k_last conditions
        cond_1  = V[k+1] < v[k+1] <= 0  # both move backwards, big block moves faster
        cond_2 = 0 < v[k+1] <= -V[k+1]  # big block moves backwards, small block will be reflected one more time
        collisions+=1
        k_last = k+1  # update k_last
        if cond_1:
            clacks = 2*k_last-1
            break
        elif cond_2:
            clacks = 2*k_last
            break
    return collisions
#     return clacks, V[:k_last+1], v[:k_last+1]

In [27]:
run_simulation(100**(6-1))

157080